In [13]:
# path for input and target data tables

diagnosis_path = '../data/diagnosis.csv'
discharge_path ='../data/discharge.csv'
edstays_path = '../data/edstays.csv'
radiology_path = '../data/radiology.csv'
triage_path = '../data/triage.csv'
target_path = '../data/discharge_target.csv'
discharge_sections_path = 'discharge_sections.csv'
radiology_sections_path = 'radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)


In [14]:
print(discharge_df['text'].iloc[0])

 
Name:  ___             Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
IV Dye, Iodine Containing Contrast Media / Oxycodone / 
cilostazol / Varenicline
 
Attending: ___.
 
Chief Complaint:
Shortness of Breath

 
Major Surgical or Invasive Procedure:
N/A

 
History of Present Illness:
Ms. ___ is a ___ female with history of 
COPD on home O2, atrial fibrillation on apixaban, hypertension, 
CAD, and hyperlipidemia who presents with shortness of breath, 
cough, and wheezing for one day.

The patient reports shortness of breath, increased cough 
productive of ___ red-flected sputum, and wheezing since 
yesterday evening.  She has been using albuterol IH more 
frequently (___) with ipratropium nebs every 4 hours with 
minimal relief. She had to increase her O2 flow up to 4L without 
significant improvement. She was currently taking 10mg of 
prednisone. She has also been taking tiotropium I

In [3]:
# segment radiology notes
def extract_text_by_subheading_radiology(text):
    headings_regex = {
        'EXAMINATION': r'(?:EXAMINATION:|EXAM:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'INDICATION':  r'(?:INDICATION:|CLINICAL INFORMATION:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'TECHNIQUE': r'(?:TECHNIQUE:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'COMPARISON': r'(?:COMPARISON)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'FINDINGS': r'(?:FINDINGS:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'IMPRESSION': r'IMPRESSION:(.*)'
    }

    extracted_text = {}

    # Iterate over each subheading and extract corresponding text
    for heading, regex in headings_regex.items():
        match = re.search(regex, text, re.DOTALL)
        if match:
            extracted_text[heading] = match.group(1).strip()
        else:
            extracted_text[heading] = None

    return extracted_text

#apply function
radiology_sections_df = radiology_df['text'].apply(extract_text_by_subheading_radiology).apply(pd.Series)

# Concatenating the columns from radiology_df into radiology_sections_df
radiology_sections_df = pd.concat([radiology_df[['subject_id', 'hadm_id']], radiology_sections_df], axis=1)

In [4]:
# segment discharge notes

def extract_text_by_subheading_discharge(text):
    
    headings_regex = {
        'CC': r'(CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:)(.*?)(?=Service:|Specialty:|Unit:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|Major Surgical or Invasive Procedure:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Service': r'(Service:|Specialty:|Unit:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Major Surgical Procedure': r'(Major Surgical or Invasive Procedure:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'HPI': r'(HPI:|History of Present Illness:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'PMH': r'(PMH:|Past Medical History:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'SOC': r'(Social History:|SOC:|SH:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'FH': r'(Family History:|PFH:|FH:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Past Surgical History': r'(Past Surgical History:|PSH:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Problem List': r'(Problem List:|Problems:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Physical Exam': r'(Physical Exam:|PE:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Medication Lists': r'(Medication lists:|Admission Medications:|Medications on Admission:|Preadmission Medication list)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Pertinent Results': r'(Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'BHC': r'(BHC:|Brief Hospital Course:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Transitional Issues': r'(Transitional Issues:|Transitional Issues|TRANSITIONAL ISSUES:)(.*?)(?=CODE STATUS:|Disposition:|Dispo:|CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Disposition': r'(Disposition:|Dispo:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Discharge Instructions': r'(Discharge Instructions:|Patient Instructions:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Followup Instructions': r'(Followup Instructions:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Discharge Diagnosis': r'(Discharge Diagnosis:|Diagnosis:)(.*?)(?=Discharge Condition:|CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:| Service:|Specialty:|Unit:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:$)',
    }

    extracted_text = {}
    for heading, regex in headings_regex.items():
        match = re.search(regex, text, re.DOTALL)
        if match:
            extracted_text[heading] = match.group(2).strip()
        else:
            extracted_text[heading] = None        

    return extracted_text

#apply function
discharge_sections_df = discharge_df['text'].apply(extract_text_by_subheading_discharge).apply(pd.Series)

# Concatenating the columns from radiology_df into radiology_sections_df
discharge_sections_df= pd.concat([discharge_df[['subject_id', 'hadm_id']], discharge_sections_df], axis=1)



In [9]:
print(target_df['discharge_instructions'].iloc[0])
print(target_df['brief_hospital_course'].iloc[0])


Dear Ms. ___,

You were admitted to ___ after you developed 
shortness of breath and wheezing at home shortly after your last 
discharge. You were treated for a COPD exacerbation and your 
breathing quickly got better. Our physical therapists evaluated 
you and recommended that you have a short stay at Pulmonary 
___ before going home to improve your breathing. 

We wish you all the best at rehab and send our condolences to 
your family on your recent loss.

It was truly a pleasure taking care of you.

Your ___ Team
Ms. ___ is a ___ female with history of 
COPD on home O2, atrial fibrillation on apixaban, hypertension, 
CAD, and hyperlipidemia who presents with shortness of breath, 
cough, and wheezing for one day. Pt recently DC'd from hospital 
for dyspnea, treated only w/nebs and steroids as not thought ___ 
true COPD exacerbation, c/f anxiety component. Pt re-admitted 
w/similar Sx, thought ___ COPD exacerbation, received nebs, 
steroids, azithromycin. Pt's wheezing, cough, SOB imp

In [5]:
print(discharge_sections_df['Transitional Issues'].iloc[0])


[] For pt's continued COPD exacerbations, recommend finishing 5d 
course of Azithromycin, 250mg qd until ___
[] Recommend extended prednisone taper for pt, 5d 40mg 
Prednisone (to finish ___ followed by 10mg taper every 5 days 
(35mg from ___, 30mg ___, etc...). 
[] Would consider PCP prophylaxis with ___ if unable to wean 
prednisone to less than 20mg daily. 
[] Pt's SOB may have an anxiety component, may benefit from 
starting SSRI in addition to home benzos already prescribed

# CONTACT: Full Code
#


In [10]:
discharge_sections_df.to_csv('discharge_sections.csv')
radiology_sections_df.to_csv('radiology_sections.csv')

In [15]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
import pandas as pd

# Load the model and tokenizer
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

def query_with_llama(question, df, columns, hadm_id):
    """
    Queries a DataFrame using the LLaMA model with data relevant to the specified HADM ID.
    
    Args:
        question (str): The question to ask the model.
        df (pd.DataFrame): The DataFrame containing the data.
        columns (list): List of columns to use for creating the context.
        hadm_id (int): The HADM ID to filter the DataFrame on.
    
    Returns:
        str: The answer from the model or an error message.
    """
    # Filter the dataframe for the specific hadm_id
    df = df[df['hadm_id'] == hadm_id]
    if df.empty:
        return "No records found for the given HADM ID."

    # Create a context string from the specified columns
    context_data = df[columns].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep=' ')
    inputs = tokenizer(question + " " + context_data, return_tensors="pt", truncation=True, max_length=1024)

    # Generate the response using the model
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model.generate(**inputs, max_length=1024)
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# Example usage (ensure 'merged_df_expanded_with_discharge' is defined in your script)
question = "Given this information from a patient's medical record and clinical notes, What major major problems did the patient have and what major treatments, surgeries or procedures did the patient receive?"
hadm_id = 24962904  # Specify the HADM ID you want to query
relevant_cols = ['HPI', 'Major Surgical Procedure']
answer = query_with_llama(question, discharge_sections_df, relevant_cols, hadm_id)
print(answer)

What major treatments, surgeries or procedures did the patient receive? Ms. ___ is a ___ female with history of 
COPD on home O2, atrial fibrillation on apixaban, hypertension, 
CAD, and hyperlipidemia who presents with shortness of breath, 
cough, and wheezing for one day.

The patient reports shortness of breath, increased cough 
productive of ___ red-flected sputum, and wheezing since 
yesterday evening.  She has been using albuterol IH more 
frequently (___) with ipratropium nebs every 4 hours with 
minimal relief. She had to increase her O2 flow up to 4L without 
significant improvement. She was currently taking 10mg of 
prednisone. She has also been taking tiotropium IH, 
theophylline, advair IH at home as prescribed. She denies sick 
contacts. She quit smoking approximately 1 month ago.

She reports an episode of chest pain in waiting room while 
sitting down, non-exertional, resolved after 2 minutes. She 
denies fever/chills, abdominal pain, nausea/vomiting, 
palpitations, and 